Downstream Analysis Pipeline: try to block (knock out) differentienly expressed genes and see if impact proliferation

Block: FOXA1, BCL2, BIRC5

In [ ]:
import maboss
import ginsim
import pandas as pd 
import numpy as np
import os
import shutil
from collections import defaultdict
from statsmodels.stats.power import TTestIndPower
import ast
import tempfile 

from functions.generate_utils.identification_patients.get_patients_sens_res import get_patients


from functions.generate_utils.create_generic_models.create_generic_patients_cfgs import create_generic_patients_cfgs_bnds

from functions.generate_utils.create_generic_models.update_phenotypes_generic_models import generic_models_update_phenotypes
from functions.generate_utils.pre_process_data.pre_process_genes import create_table_rna_seq_patients, process_genes

from functions.generate_utils.create_person_models.tailor_cfgs_patients import personalized_patients_genes_cfgs
from functions.generate_utils.create_person_models.tailor_bnd_mutations import personalized_patients_mutations_bnds
from functions.generate_utils.create_person_models.tailor_bnd_cnv import tailor_bnd_cnv_cm


from functions.analysis_utils.MaBoSS_simulation.maboss_phenotype_patient import compute_phenotype_table, compute_phenotype_mean_group_validation


# # from pre_process_profiles_table_data_lung import create_genes_patients

from functions.analysis_utils.stats.stats_proba import compute_mannwhitneyu_test_means, compute_kruskal, compute_power_calculation
from functions.analysis_utils.results_MaBoSS_visualization.boxplot_phenotype import create_boxplot
from functions.analysis_utils.results_MaBoSS_visualization.create_phenotypes_patients_table import vizualise_table_phenotype_condition
from functions.analysis_utils.results_MaBoSS_visualization.patients_ids_phenotype_table import create_combined_table_patients
from functions.analysis_utils.gene_enrichment.genes_signature import compute_genes_mean_signature
from functions.generate_utils.pre_process_data.pre_process_cnv import preprocess_cnv
from functions.generate_utils.pre_process_data.pre_process_montagud_nodes import process_montagud_nodes


from functions.analysis_utils.genes_intervention.pers_interventions import tailor_bnd_genes_intervention

Step 0: Import data 

In [17]:
# Import Data

annotations_models = pd.read_csv('data/model_list_20250407.csv')
mutations_data = pd.read_csv('data/mutations_all_20250318.csv')
drug_data = pd.read_csv('data/drug_sensitivity.csv')

montagud_data = (
    pd.read_csv('data/Montagud_inter_nodes_data.csv', header=1)
    .loc[:, ['Target node', 'Interaction type', 'Source']])
rna_seq_data = pd.read_csv('data/rnaseq_merged/rnaseq_merged_20250117.csv')
cnv_data = pd.read_csv('data/cellmodel_data/cnv_summary_20250207.csv')


models_depmap_annotation = pd.read_csv('data/cellmodel_data/model_list_20250423.csv')
mutations_data_depseq = pd.read_csv('data/depmap_data/OmicsSomaticMutations.csv')



/var/folders/83/pqwsm5cn7vscbq8l3fnf30qr0000gn/T/ipykernel_42538/4029099413.py:15: DtypeWarning: Columns (22,50,56,57,58,59,61) have mixed types. Specify dtype option on import or set low_memory=False.
  mutations_data_depseq = pd.read_csv('data/depmap_data/OmicsSomaticMutations.csv')


Step 1: Identify the best drug to keep (the one with most resistant and sensitive, and with big variance) 
        & specify the phenotypes of our model

In [18]:
tissue_remove = 'Haematopoietic and Lymphoid'
# drugs_df = identify_drug(drug_data, annotations_models, tissue_remove)

In [19]:
drug_interest = 'Refametinib' #Pictilisib, 'Avagacestat' AZD8931 
tissue_interest = 'PAN_CANCER'


type_models = 'genes_models'


# models inputs
inputs_list = ['EGF', 'FGF', 'TGFB', 'Androgen', 'Hypoxia', 'Nutrients', 'Carcinogen', 'Acidosis', 'TNF', 'SPOP']


# models outputs
phenotype_interest = ["Proliferation","Invasion","DNA_Repair","Migration","Apoptosis"]



Step 2: Create the folders where the models are saved and where the results will be saved

In [20]:

# folder where the results will be saved
folder_result = f'results/{drug_interest}_{tissue_interest}_intervention'

folder_pers_models=f'models/personalized_boolean_{drug_interest}_{tissue_interest}_intervention'


# Generic model that will be duplicated to create personalized models
folder_generic_models = f'{folder_pers_models}/generic_models'


src_dir_generic_models = f'models/prostate/generic/{type_models}'

# copy the generic model from the models to validation generic models

folder_generic_models_cfg = f'{folder_generic_models}/Montagud2022_Prostate_Cancer.cfg'
folder_generic_models_bnd = f'{folder_generic_models}/Montagud2022_Prostate_Cancer.bnd'


folder_res_results_single_input = f'{folder_result}/resistant_results/only_gene_expression/single_input_on'
folder_sens_results_single_input = f'{folder_result}/sensitive_results/only_gene_expression/single_input_on'



folder_result_resistant = f'{folder_res_results_single_input}/phenotype_distribution_patients'
folder_result_sensitive = f'{folder_sens_results_single_input}/phenotype_distribution_patients'

folder_result_res_sen = f'{folder_result}/sensitive_resistant_results/genes_diff_expressed'


generic_pers_models_folder_res = f"{folder_pers_models}/resistant_patient/generic_models"
generic_pers_models_folder_sens = f"{folder_pers_models}/sensitive_patient/generic_models"

folder_models_res = f'{folder_pers_models}/resistant_patient/personalized_boolean_modified/models_gene_expression'
folder_models_sens = f'{folder_pers_models}/sensitive_patient/personalized_boolean_modified/models_gene_expression'




results_mean_folder_res = f"{folder_res_results_single_input}/phenotype_mean"
results_mean_folder_sens = f"{folder_sens_results_single_input}/phenotype_mean"



os.makedirs(folder_result_resistant, exist_ok=True)
os.makedirs(folder_result_sensitive, exist_ok=True)
os.makedirs(folder_result_res_sen, exist_ok=True)

os.makedirs(folder_generic_models, exist_ok=True)
os.makedirs(folder_models_res, exist_ok=True)
os.makedirs(folder_models_sens, exist_ok=True)

os.makedirs(results_mean_folder_res, exist_ok=True)
os.makedirs(results_mean_folder_res, exist_ok=True)

for filename in os.listdir(src_dir_generic_models):
    src_file = os.path.join(src_dir_generic_models, filename)
    dst_file = os.path.join(folder_generic_models, filename)
    if os.path.isfile(src_file):
        shutil.copy(src_file, dst_file)

        



Step 3: Get the n top patients Ids

In [21]:
# top_resistant_ids, top_sensitive_ids= get_patients(drug_data, annotations_models, drug_interest, tissue_interest)
top_resistant_ids, top_sensitive_ids, drug_tissue_data= get_patients(20, drug_data, annotations_models, drug_interest, tissue_interest = None, tissue_remove = tissue_remove)
patients_ids = top_sensitive_ids + top_resistant_ids

/Users/romane/repos/icr/Attractor-Resistance/identification_patients/get_patients_sens_res.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  annotations_models_filtered.rename(


Step 4: Pre-process data- Montagud nodes, rna-seq (filtered with patients IDs and Montagud nodes)

In [22]:
# Pre-process Montagud nodes
nodes_to_remove = ['FUSED_EVENT', 'AR_ERG']
montagud_nodes = process_montagud_nodes(montagud_data, node_to_remove=nodes_to_remove)

In [23]:
# Pre-process RNA seq data (keep only montagud_nodes and patients ids) and create table for high and low expression
rna_seq_data_filtered = process_genes(patients_ids, montagud_nodes, rna_seq_data)
table_rna_seq_patients = create_table_rna_seq_patients(rna_seq_data_filtered)

/Users/romane/repos/icr/Attractor-Resistance/pre_process_data/pre_process_genes.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rna_seq_data.rename(columns={"gene_symbol": "gene_name"}, inplace=True)


In [24]:
# Pre-process patients (keep only the patients id with gene expression data)

top_resistant_ids = list(set(table_rna_seq_patients.index) & set(top_resistant_ids))
top_sensitive_ids= list(set(table_rna_seq_patients.index) & set(top_sensitive_ids))

patients_ids = top_resistant_ids + top_sensitive_ids

In [25]:
def collect_group_data(group_folder_path):
    combined_data = defaultdict(lambda: defaultdict(list))

    for file in os.listdir(group_folder_path):
        if file.startswith("_SID") and file.endswith(".csv"):
            file_path = os.path.join(group_folder_path, file)
            df = pd.read_csv(file_path, index_col=0)

            for input_name in df.index:
                for phenotype in df.columns:
                    value = df.at[input_name, phenotype]
                    combined_data[input_name][phenotype].append(float(value))

    result_df = pd.DataFrame.from_dict(combined_data, orient='index')
    result_df.to_csv(os.path.join(group_folder_path, "combined_results.csv"))

    return result_df

In [26]:
genes_enrichment_cond_pheno = pd.read_csv('results/genes_enrichment_egf_prolif.csv')
genes_enrichment_cond_pheno = genes_enrichment_cond_pheno.rename(columns={'Unnamed: 0': 'gene_name'})

genes_enrichment_cond_pheno = genes_enrichment_cond_pheno.sort_values(by='diff_means', ascending = False)

# Select top 8 genes based on the 'diff_means' column
if isinstance(genes_enrichment_cond_pheno['gene_name'].iloc[:8], str):
    top_diff_genes = [genes_enrichment_cond_pheno['gene_name'].iloc[:8]]
else:
    top_diff_genes = list(genes_enrichment_cond_pheno['gene_name'].iloc[:8])

In [27]:
print(type(top_diff_genes))
print(type(top_diff_genes[1]))

<class 'list'>
<class 'str'>


In [28]:
genes_to_target = []

print("top_diff_genes type:", type(top_diff_genes))
print("top_diff_genes:", top_diff_genes)


for gene in top_diff_genes:
    print(gene)
    print(type(gene))
    # 1. Create a temporary directory for this gene's models
    with tempfile.TemporaryDirectory() as temp_res_dir, tempfile.TemporaryDirectory() as temp_sens_dir:

        # 2. Copy the original models into the temp directory
        shutil.copytree(folder_models_res, temp_res_dir, dirs_exist_ok=True)
        shutil.copytree(folder_models_sens, temp_sens_dir, dirs_exist_ok=True)

        tailor_bnd_genes_intervention(gene, top_resistant_ids, temp_res_dir, tissue_interest)
        tailor_bnd_genes_intervention(gene, top_resistant_ids, temp_sens_dir, tissue_interest)


        for patient in top_resistant_ids:
            compute_phenotype_table(folder_result_resistant, temp_res_dir, patient, inputs_list, phenotype_interest, drug_interest)

        for patient in top_sensitive_ids:
            compute_phenotype_table(folder_result_sensitive, temp_sens_dir, patient, inputs_list, phenotype_interest, drug_interest)



        df_res_combined = collect_group_data(folder_result_resistant)
        df_sens_combined = collect_group_data(folder_result_sensitive)
        
        df_res_combined = pd.read_csv(f'{folder_result_resistant}/combined_results.csv', index_col=0)  # index: inputs, columns: phenotypes
        df_sens_combined = pd.read_csv(f'{folder_result_sensitive}/combined_results.csv', index_col=0)

        patient_res_values =  pd.read_csv(f'{folder_result_resistant}/combined_results.csv')
        patient_sens_values = pd.read_csv(f'{folder_result_sensitive}/combined_results.csv')
        compute_mannwhitneyu_test_means(folder_result,patient_res_values, patient_sens_values, drug_interest)

        data_greater_side = pd.read_csv(f'{folder_result}/sensitive_resistant_results/p_values_df_mannwhitneyu_greater_sign_{drug_interest}.csv')
        data_less_side = pd.read_csv(f'{folder_result}/sensitive_resistant_results/p_values_df_mannwhitneyu_less_sign_{drug_interest}.csv')
        data_both_side = pd.read_csv(f'{folder_result}/sensitive_resistant_results/p_values_df_mannwhitneyu_two_sides_{drug_interest}.csv')

        signif_df = pd.concat([data_greater_side, data_less_side, data_both_side], axis=0)

        if ('Proliferation' not in data_greater_side['Phenotype'].values):
            genes_to_target.append(gene)


print(genes_to_target)



top_diff_genes type: <class 'list'>
top_diff_genes: ['FOXA1', 'FGFR3', 'JUN', 'FRS2', 'BCL2', 'E2F1', 'APAF1', 'EZH2']
FOXA1
<class 'str'>
🔍 Processing patient SIDM00892, gene: FOXA1
FOXA1 node found. Replacing...
SIDM00892_Refametinib: CNV — nodes modified
🔍 Processing patient SIDM01056, gene: FOXA1
FOXA1 node found. Replacing...
SIDM01056_Refametinib: CNV — nodes modified
🔍 Processing patient SIDM00719, gene: FOXA1
FOXA1 node found. Replacing...
SIDM00719_Refametinib: CNV — nodes modified
🔍 Processing patient SIDM01120, gene: FOXA1
FOXA1 node found. Replacing...
SIDM01120_Refametinib: CNV — nodes modified
🔍 Processing patient SIDM00255, gene: FOXA1
FOXA1 node found. Replacing...
SIDM00255_Refametinib: CNV — nodes modified
🔍 Processing patient SIDM00870, gene: FOXA1
FOXA1 node found. Replacing...
SIDM00870_Refametinib: CNV — nodes modified
🔍 Processing patient SIDM00770, gene: FOXA1
FOXA1 node found. Replacing...
SIDM00770_Refametinib: CNV — nodes modified
🔍 Processing patient SIDM007

KeyboardInterrupt: 

Personalize the cfg files with the "simulated KO" of FOXA1, BCL2, and BIRC5

In [ ]:
# tailor_bnd_genes_intervention(top_diff_genes, top_resistant_ids, folder_models_res, tissue_interest)

Step 7: compute the phenotype distribution 

In [ ]:
# Compute phenotype distribution for each patient

# for patient in top_resistant_ids:
#     compute_phenotype_table(folder_result_resistant, folder_models_res, patient, inputs_list, phenotype_interest, drug_interest)
    
# for patient in top_sensitive_ids:
#     compute_phenotype_table(folder_result_sensitive, folder_models_sens, patient, inputs_list, phenotype_interest, drug_interest)
    

In [ ]:
# df_res_combined = collect_group_data(folder_result_resistant)
# df_sens_combined = collect_group_data(folder_result_sensitive)


In [ ]:
# stats test between the two combined dataframes (sensitive vs resistant)

# res_combined_path = "results/Refametinib_PAN_CANCER/resistant_results/only_gene_expression/single_input_on/phenotype_distribution_patients/combined_results.csv"
# sens_combined_path = "results/Refametinib_PAN_CANCER/sensitive_results/only_gene_expression/single_input_on/phenotype_distribution_patients/combined_results.csv"

# df_res_combined = pd.read_csv(f'{folder_result_resistant}/combined_results.csv', index_col=0)  # index: inputs, columns: phenotypes
# df_sens_combined = pd.read_csv(f'{folder_result_sensitive}/combined_results.csv', index_col=0)

In [ ]:
# patient_res_values =  pd.read_csv(f'{folder_result_resistant}/combined_results.csv')
# patient_sens_values = pd.read_csv(f'{folder_result_sensitive}/combined_results.csv')
# compute_mannwhitneyu_test_means(folder_result,patient_res_values, patient_sens_values, drug_interest)

In [ ]:
# data_greater_side = pd.read_csv(f'{folder_result}/sensitive_resistant_results/p_values_df_mannwhitneyu_greater_sign_{drug_interest}.csv')
# data_less_side = pd.read_csv(f'{folder_result}/sensitive_resistant_results/p_values_df_mannwhitneyu_less_sign_{drug_interest}.csv')
# data_both_side = pd.read_csv(f'{folder_result}/sensitive_resistant_results/p_values_df_mannwhitneyu_two_sides_{drug_interest}.csv')

# signif_df = pd.concat([data_greater_side, data_less_side, data_both_side], axis=0)

In [ ]:
create_boxplot(df_res_combined, df_sens_combined, data_both_side)

Compute attractors phenotype mean and power calculation to count the nb of patients needed in each group

In [ ]:
conditions = df_res_combined.index
phenotypes = df_res_combined.columns
mean_df_res_combined = pd.DataFrame(index=conditions, columns=phenotypes)


for condition, values in df_res_combined.iterrows():
    for phenotype in df_res_combined.columns:

        df_res_values = df_res_combined.loc[condition][phenotype]
        df_res_values = ast.literal_eval(df_res_values)
        mean = sum(df_res_values)/ len(df_res_values)
        mean_df_res_combined.loc[condition][phenotype] = mean
print(mean_df_res_combined)




conditions = df_sens_combined.index
phenotypes = df_sens_combined.columns
mean_df_sens_combined = pd.DataFrame(index=conditions, columns=phenotypes)


for condition, values in df_sens_combined.iterrows():
    for phenotype in df_sens_combined.columns:

        df_res_values = df_sens_combined.loc[condition][phenotype]
        df_res_values = ast.literal_eval(df_res_values)
        mean = sum(df_res_values)/ len(df_res_values)
        mean_df_sens_combined.loc[condition][phenotype] = mean
print(mean_df_sens_combined)

Power calculation for every condition-phenotype pairs

In [ ]:
# # power calculation 

# nb_patients_required = pd.DataFrame(index=conditions, columns=phenotypes)

# for condition, values in df_res_combined.iterrows():
#     for phenotype in df_res_combined.columns:


#         data_res =  ast.literal_eval(df_res_combined.loc[condition][phenotype])
#         data_sens = ast.literal_eval(df_sens_combined.loc[condition][phenotype])

#         res_patients_mean = np.mean(data_res)
#         sens_patients_mean = np.mean(data_sens)

#         res_patients_std = np.std(data_res)
#         sens_patients_std = np.std(data_sens)

#         n_res_group = len(data_res)
#         n_sens_group = len(data_sens)
#         sd_pooled = np.sqrt(
#             ((n_res_group - 1) * res_patients_std**2 + (n_sens_group - 1) * sens_patients_std**2)
#             / (n_res_group + n_sens_group - 2)
#         )

#         d = res_patients_mean -sens_patients_mean/sd_pooled

#         analysis = TTestIndPower()
#         sample_size = analysis.solve_power(effect_size=d, power=0.8, alpha=0.05, alternative='two-sided')
#         nb_patients_required.loc[condition][phenotype] = sample_size

# print(nb_patients_required)
# print(nb_patients_required[nb_patients_required <= 20])





Results: using a T-test with alpha of 0.05 we will have 80% power to detect an effect size of 0.62 * StDev (ie effect > 10) with a minimum study size of 19 patients per group.

In [ ]:
significant_kruskal_results = compute_kruskal(inputs_list, phenotype_interest, df_res_combined, df_sens_combined)

Step 7: Compute stats test between two mean datasets

In [ ]:
# patient_res_stats_values = pd.read_csv(f'{folder_result}/resistant_results/only_gene_expression/single_input_on/patients_resistant_values_stats_{drug_interest}.csv')
# patient_sens_stats_values = pd.read_csv(f'{folder_result}/sensitive_results/only_gene_expression/single_input_on/patients_sensitive_values_stats_{drug_interest}.csv')

patient_res_stats_values =  pd.read_csv(f'{folder_result_resistant}/combined_results.csv')
patient_sens_stats_values = pd.read_csv(f'{folder_result_sensitive}/combined_results.csv')
compute_mannwhitneyu_test_means(folder_result,patient_res_stats_values, patient_sens_stats_values, drug_interest)

Step 8: Vizualise the boxplot of phenotype distribution output

In [ ]:
# patient_res_values = pd.read_csv(f'{folder_result}/resistant_results/only_gene_expression/single_input_on/patients_resistant_values_stats_{drug_interest}.csv')
# patient_sens_values = pd.read_csv(f'{folder_result}/sensitive_results/only_gene_expression/single_input_on/patients_sensitive_values_stats_{drug_interest}.csv')
#data_greater_side = pd.read_csv(f'{folder_result}/sensitive_resistant_results/p_values_df_mannwhitneyu_greater_sign_{drug_interest}.csv')


data_greater_side = pd.read_csv(f'{folder_result}/sensitive_resistant_results/p_values_df_mannwhitneyu_greater_sign_Refametinib.csv')
data_less_side = pd.read_csv(f'{folder_result}/sensitive_resistant_results/p_values_df_mannwhitneyu_less_sign_Refametinib.csv')
data_both_side = pd.read_csv(f'{folder_result}/sensitive_resistant_results/p_values_df_mannwhitneyu_two_sides_Refametinib.csv')

signif_df = pd.concat([data_greater_side, data_less_side, data_greater_side], axis=0)

In [ ]:
create_boxplot(patient_res_stats_values, patient_sens_stats_values, signif_df)


Step 9: create table of patients with conditions- phenotype

In [ ]:
res_tables_path = f'{folder_res_results_single_input}/phenotype_distribution_patients'
sens_tables_path = f'{folder_sens_results_single_input}/phenotype_distribution_patients'

In [ ]:
patients_phenot_table = create_combined_table_patients(res_tables_path, sens_tables_path, folder_result)

# patients_phenot_table = create_table_patients_phenotypes(folder_result, dir_res_data, dir_sens_data)

Step 10: Create heatmap figure 

In [ ]:
# patient_resistant_mean = pd.read_csv(f'{folder_res_results_single_input}/patients_resistant_df_mean_{drug_interest}.csv')
# patient_sensitive_mean = pd.read_csv(f'{folder_sens_results_single_input}/patients_sensitive_df_mean_{drug_interest}.csv')
# vizualise_table_phenotype_condition(folder_result, patient_resistant_mean, patient_sensitive_mean)

Step 11: Identify genes differently expressed in the patients with high 

In [ ]:
# identify the condition (input) and phenotype for which we observe a significant increase in resistant patients

greater_sign_results = pd.read_csv(f'{folder_result}/sensitive_resistant_results/p_values_df_mannwhitneyu_greater_sign_Refametinib.csv')
conditions_phenotypes_df = greater_sign_results[['Condition', 'Phenotype']]
conditions_phenotypes_df

In [ ]:
patients_phenot_table = pd.read_csv(f'{folder_result}/sensitive_resistant_results/patients_phenot_table.csv')

In [ ]:
# identify the patients id that show this condition-phenotype pair 

# save every gene enrichment dataframe in a dictionary of index condtion_phenotype
indexes_list = []
for condition, phenotype in conditions_phenotypes_df.values:
    indexes_list.append(f'{condition}_{phenotype}')
results_genes_enrich = {key: None for key in indexes_list}

for condition, phenotype in conditions_phenotypes_df.values:
    genes_stats_results, resistant_group_ids, sensitive_group_ids = compute_genes_mean_signature(rna_seq_data_filtered, 
    folder_result, phenotype, condition, patients_phenot_table, top_resistant_ids, top_sensitive_ids)
    if isinstance(genes_stats_results, pd.DataFrame):
        results_genes_enrich[f'{condition}_{phenotype}'] = genes_stats_results
        

Step 11: check there is not correlation between phenotype distribution and cancer type 


Gene enrichment for EGF Proliferation (as required 19 patients in each group and statistically relevant with Mannwhiteney test)

In [ ]:
results_egf_prolif = results_genes_enrich['EGF_Proliferation']
results_egf_prolif['diff_means'] = results_egf_prolif['Group Resistant Mean'] - results_egf_prolif['Group Sensitive Mean'] 
results_egf_prolif.sort_values(by = 'P-value', ascending=True).head()

results_egf_prolif.sort_values(by = 'diff_means', ascending=False)
results_egf_prolif.to_csv("results/Refametinib_PAN_CANCER_intervention/sensitive_resistant_results/genes_enrichment_egf_prolif.csv", index=True)


Results:
- FOXA1: TF, key roles in hormone receptor signaling and modulates tumor growth and metastatic potential
- FGFR3: receptor tyrosine kinase for FGF ligands, promoting cell prolif (frequently overexpressed in cancers)
- FRS2: overexpressed in several cancers, enhancing proliferative/ migratory responses
- BCL2, BIRC5: anti-apoptotic protein -> promotes cell survival by inhibiting programmed cell death pathways; for overepxressed in cancer to evade apoptois.
- EZH2: silencing TSG via H3K27 trimethylation, overexpression linked to aggressive, proliferative cancers

In [ ]:
len(resistant_group_ids)
len(sensitive_group_ids)

Identify if this FOXA1 overexpression is not related to tissue 

In [ ]:
resistant_group_ids_models = annotations_models[annotations_models['model_id'].isin(resistant_group_ids)].copy()
sensitive_group_ids_models = annotations_models[annotations_models['model_id'].isin(sensitive_group_ids)].copy()

In [ ]:
resistant_counts = resistant_group_ids_models['tissue'].value_counts()
total_counts = annotations_models['tissue'].value_counts()
fraction_resistant = resistant_counts / total_counts
print(fraction_resistant[fraction_resistant.notna()])


sensitive_counts = sensitive_group_ids_models['tissue'].value_counts()
total_counts_sens = annotations_models['tissue'].value_counts()
fraction_sensitive = sensitive_counts / total_counts_sens
print(fraction_sensitive[fraction_sensitive.notna()])


Result: As the ratio do not vary that much, the results observed do not seem to be tissue-dependent

In [ ]:
resistant_counts = resistant_group_ids_models['gender'].value_counts()
total_counts = annotations_models['gender'].value_counts()
fraction_resistant = resistant_counts / total_counts
print(fraction_resistant[fraction_resistant.notna()])


sensitive_counts = sensitive_group_ids_models['gender'].value_counts()
total_counts_sens = annotations_models['gender'].value_counts()
fraction_sensitive = sensitive_counts / total_counts_sens
print(fraction_sensitive[fraction_sensitive.notna()])

Result: As the ratio do not vary that much, the results observed do not seem to be gender-dependent

In [ ]:
# power calculation for FOXA1

foxa1_data_res = list(rna_seq_data_filtered[
    (rna_seq_data_filtered['model_id'].isin(resistant_group_ids)) &
    (rna_seq_data_filtered['gene_symbol'] == 'FOXA1')
]['rsem_tpm'])


foxa1_data_sens = list(rna_seq_data_filtered[
    (rna_seq_data_filtered['model_id'].isin(sensitive_group_ids)) &
    (rna_seq_data_filtered['gene_symbol'] == 'FOXA1')
]['rsem_tpm'])


res_patients_mean = np.mean(foxa1_data_res)
sens_patients_mean = np.mean(foxa1_data_sens)


res_patients_std = np.std(foxa1_data_res)
sens_patients_std = np.std(foxa1_data_sens)

n_res_group = len(resistant_group_ids)
n_sens_group = len(sensitive_group_ids)

sd_pooled = np.sqrt(
    ((n_res_group - 1) * res_patients_std**2 + (n_sens_group - 1) * sens_patients_std**2)
    / (n_res_group + n_sens_group - 2)
)

d = res_patients_mean -sens_patients_mean/sd_pooled


# Set parameters
effect_size = d              # Cohen's d
alpha = 0.05                 # Significance level
power = 0.80                 # Desired power
alternative = 'two-sided'    # Equivalent to "two.sample" in R

# Initialize power analysis object
analysis = TTestIndPower()

# Compute required sample size per group
sample_size = analysis.solve_power(effect_size=effect_size, power=power, alpha=alpha, alternative=alternative)

print(f"Required sample size per group: {float(sample_size):.2f}")

